<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifiers/ADAPMIT/CPU_ADAPMIT_SentenceT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [2]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 15:18:58] [setup] RAM Tracking...
[codecarbon INFO @ 15:18:58] [setup] GPU Tracking...
[codecarbon INFO @ 15:18:58] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:18:58] [setup] CPU Tracking...
[codecarbon WARNING @ 15:18:58] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 15:18:59] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 15:18:59] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 15:18:59] >>> Tracker's metadata:
[codecarbon INFO @ 15:18:59]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 15:18:59]   Python version: 3.10.12
[codecarbon INFO @ 15:18:59]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 15:18:59]   Available RAM : 12.675 GB
[codecarbon INFO @ 15:18:59]   CPU count: 2
[codecarbon INFO @ 15:18:59]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 15:18:59]   GPU c

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

[codecarbon INFO @ 15:20:44] Energy consumed for RAM : 0.000139 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 15:20:44] Energy consumed for all GPUs : 0.000285 kWh. Total GPU Power : 9.702970539528101 W
[codecarbon INFO @ 15:20:44] Energy consumed for all CPUs : 0.001244 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:20:44] 0.001668 kWh of electricity used since the beginning.


# Manual Inputs


In [6]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
train_file="policy_classification_train.json"
test_file="policy_classification_test.json"


### Define labels ##################
label_names= ['AdaptationLabel','MitigationLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "BAAI/bge-base-en-v1.5"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "ADAPMIT-multilabel-bge"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=4.08e-5   #4.04e-5
batch_size = 16
num_train_epochs= 4
weight_decay=0.021
warmup_steps = 300
gradient_accumulation_steps = 1

# Automated

## Dataset

In [7]:
# Load Dataset from Hugging Face
data_files = {"train": train_file, "test": test_file}
dataset = load_dataset(repo_id, data_files=data_files)

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = dataset['train'].to_pandas()
test_df =dataset['test'].to_pandas()

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{0: 'AdaptationLabel', 1: 'MitigationLabel'}


In [8]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 12538
0 . AdaptationLabel : 5439
1 . MitigationLabel : 6659

 Classes Representation in Test Dataset: 1190
0 . AdaptationLabel : 533
1 . MitigationLabel : 604


## Model

In [9]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

[codecarbon INFO @ 15:21:14] Energy consumed for RAM : 0.000179 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 15:21:14] Energy consumed for all GPUs : 0.000366 kWh. Total GPU Power : 9.686530809730378 W
[codecarbon INFO @ 15:21:14] Energy consumed for all CPUs : 0.001598 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:21:14] 0.002142 kWh of electricity used since the beginning.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at BAAI/bge-base-en-v1.5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Training data: 12538
Validation data: 1190


Map:   0%|          | 0/12538 [00:00<?, ? examples/s]

Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

Map:   0%|          | 0/12538 [00:00<?, ? examples/s]

[codecarbon INFO @ 15:21:29] Energy consumed for RAM : 0.000198 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 15:21:29] Energy consumed for all GPUs : 0.000456 kWh. Total GPU Power : 21.819642986884812 W
[codecarbon INFO @ 15:21:29] Energy consumed for all CPUs : 0.001775 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:21:29] 0.002430 kWh of electricity used since the beginning.


Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

## Metrics

In [11]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [12]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "cosine",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

# Training

In [13]:
multi_trainer =  Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)


[codecarbon INFO @ 15:21:35] [setup] RAM Tracking...
[codecarbon INFO @ 15:21:35] [setup] GPU Tracking...
[codecarbon INFO @ 15:21:35] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:21:35] [setup] CPU Tracking...
[codecarbon WARNING @ 15:21:35] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 15:21:37] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 15:21:37] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 15:21:37] >>> Tracker's metadata:
[codecarbon INFO @ 15:21:37]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 15:21:37]   Python version: 3.10.12
[codecarbon INFO @ 15:21:37]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 15:21:37]   Available RAM : 12.675 GB
[codecarbon INFO @ 15:21:37]   CPU count: 2
[codecarbon INFO @ 15:21:37]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 15:21:37]   GPU c

In [ ]:
multi_trainer.train()

In [15]:
tracker.stop()

[codecarbon INFO @ 16:21:32] Energy consumed for RAM : 0.004949 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 16:21:32] Energy consumed for all GPUs : 0.069669 kWh. Total GPU Power : 32.01169923662747 W
[codecarbon INFO @ 16:21:32] Energy consumed for all CPUs : 0.044292 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 16:21:32] 0.118909 kWh of electricity used since the beginning.


0.0415262334928358

In [16]:
multi_trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1709652097.502e8c615647.701.0:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ppsingh/ADAPMIT-multilabel-bge/commit/37cb06530ea9e0c1c5107c8c94cceaa37dd23c1e', commit_message='End of training', commit_description='', oid='37cb06530ea9e0c1c5107c8c94cceaa37dd23c1e', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
log_results = multi_trainer.state.log_history.copy()
logs = []
while len(log_results)>1:

    a = log_results[0]
    b = log_results[1]
    logs.append({**a,**b})
    log_results.pop(0)
    log_results.pop(0)

results_df = pd.DataFrame(logs)
results_df

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_precision-micro,eval_precision-samples,eval_precision-weighted,eval_recall-micro,eval_recall-samples,eval_recall-weighted,eval_f1-micro,eval_f1-samples,eval_f1-weighted,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.3368,1.823478,0.000038,1.0,784,0.291678,0.865112,0.844958,0.866377,0.913808,0.854202,0.913808,0.888794,0.843697,0.888979,31.1231,38.235,2.410
1,0.1807,1.627960,0.000024,2.0,1568,0.254919,0.909170,0.864286,0.909403,0.915567,0.857143,0.915567,0.912358,0.857143,0.912342,31.1453,38.208,2.408
2,0.0955,1.378113,0.000007,3.0,2352,0.298832,0.906897,0.865966,0.907235,0.925242,0.865546,0.925242,0.915977,0.861345,0.915979,31.1541,38.197,2.407
3,0.0495,3.736454,0.000000,4.0,3136,0.310057,0.905822,0.864706,0.905845,0.930519,0.869328,0.930519,0.918004,0.862185,0.918008,31.0424,38.335,2.416


# Predict and Evaluate

## Trainer

In [18]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.50
y_pred = (y_prob>=thresh).bool()
y_true = y_true.bool()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [19]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

AdaptationLabel
[[608  49]
 [ 38 495]] 

MitigationLabel
[[525  61]
 [ 41 563]] 



In [20]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,AdaptationLabel,0.909926,0.928705,0.919220,533.0
1,MitigationLabel,0.902244,0.932119,0.916938,604.0
2,micro avg,0.905822,0.930519,0.918004,1137.0
3,macro avg,0.906085,0.930412,0.918079,1137.0
4,weighted avg,0.905845,0.930519,0.918008,1137.0
5,samples avg,0.864706,0.869328,0.862185,1137.0
